In [0]:
from pyspark.sql.functions import col, regexp_replace

# 1. Ensure schema exists
spark.sql("CREATE DATABASE IF NOT EXISTS lakehouse.silver")

# 2. Read Bronze table
bronze_df = spark.table("lakehouse.bronze.erp_loc")

# 3. Clean data (equivalent à DELETE WHERE CID IS NULL)
bronze_clean_df = bronze_df.filter(col("CID").isNotNull())

# 4. Transformations
silver_df = (
    bronze_clean_df
    .withColumn("Customer_id", col("CID").cast("string"))
    .withColumn("CUST_ID", regexp_replace(col("CID"), "-", ""))
    .withColumn("Country", col("CNTRY").cast("string"))
)

# 5. Write to Silver layer
(
    silver_df
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("lakehouse.silver.erp_locationn")
)

print("Silver ERP location table created successfully")
